# Data structure of a (pandapower) network
This notebook shows the data structure behind the LV grid. It aims to give the reader a better understanding of the LV networks.

In [ ]:
import warnings

from plotting.plot_networks import *
from src.GridGenerator import GridGenerator

warnings.filterwarnings('ignore')

Choose PLZ and network to show graph for:

In [ ]:
plz = "80803"
kcid = 1
bcid = 17

## Recall
Remember the network visualisation from the 'map visualisation example' notebook.

In [ ]:
plot_contextily(plz=plz, bcid=bcid, kcid=kcid)

## Interactive Pandapower Graphs
The generated LV grids are saved on the database in the table `grids` as pandapower networks. 
See documentation of pandapower: https://pandapower.readthedocs.io/en/latest/index.html
Hover over the following visualisation to find out more about the components.

In [ ]:
plot_simple_grid(plz=plz, kcid=kcid, bcid=bcid)

You will find that there are different kind of buses:
- Consumer Nodebuses which are house connections. Each consumer has one or multiple loads which define the power demand.
- Connection Nodebus: everywhere the cable type changes there is a connection nodebus.
- MVbus: connection to the MV grid
- LVbus: transformer
- Lines: cables of different types which have different properties like resistance, diameter, max current.

The network can also be plotted on a mapbox basemap. Feel free to zoom in with ctr+scroll.

In [ ]:
plot_grid_on_map(plz=plz, kcid=kcid, bcid=bcid)

## Pandapower Network Properties
The properties of the net shown above can be explored further:

In [ ]:
gg = GridGenerator(plz=plz)
net = gg.dbc.read_net(plz=plz, kcid=kcid, bcid=bcid)
pg = gg.dbc
net

the individual properties can be retrieved as a pandas dataframe using the net.[property] notation, eg.:

In [ ]:
df_trafo = net.trafo
df_trafo

This is the information about the transformer.

The bus data and bus geodata can be merged into one dataframe:

In [ ]:
df_bus = pd.concat([net.bus, net.bus_geodata], axis=1)
df_bus.head()

next we save the loads in the network:

In [ ]:
df_load = net.load
df_load.head()

We filter the consumer buses:

In [ ]:
consumers = net.bus
consumers['consumer_bus'] = consumers['name'].str.contains("Consumer Nodebus")
consumers = consumers[consumers['consumer_bus']]
consumers

In [ ]:
consumers['zone'].unique()


There are different types of consumers: see Ye 2022 p. 15
- SFH: single family home
- MFH: multi family home
- TH: terrace houses
- AB: apartment blocks
- public buildings
They have different number of households. 

line_geodata contains the geo information about the cables

In [ ]:
net.line_gdf = gpd.GeoDataFrame(net.line.copy(), geometry=net.line_geodata.coords.map(linestrings), crs="EPSG:4326")
net.line_gdf.head()